## load model

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq
last_checkpoint_49 = "./results/checkpoint-22149"

tokenizer = T5Tokenizer.from_pretrained(last_checkpoint_49)
model = T5ForConditionalGeneration.from_pretrained(last_checkpoint_49, device_map="auto")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

c:\Users\User\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## load dataset

In [2]:
from datasets import load_dataset


ds = load_dataset("keivalya/MedQuad-MedicalQnADataset")
ds

DatasetDict({
    train: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 16407
    })
})

## split the dataset

In [3]:
ds=ds['train'].train_test_split(test_size=0.1)
ds

DatasetDict({
    train: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 14766
    })
    test: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 1641
    })
})

## processing dataset

In [4]:
prefix = "Please answer this medical related question: "

# Define the preprocessing function

def preprocess_function(examples):

   inputs = [prefix + doc for doc in examples["Question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
  
   labels = tokenizer(text_target=examples["Answer"], 
                      max_length=512,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

tokenized_dataset = ds.map(preprocess_function, batched=True)

Map: 100%|██████████| 1641/1641 [00:01<00:00, 1407.40 examples/s]


## compute_metrics

In [5]:
import nltk
import evaluate
import numpy as np

nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

## fine-tuning

In [6]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,  
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=2, 
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_steps=20,
    save_total_limit=3,
    predict_with_generate=True,
    push_to_hub=False,
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()



c:\Users\User\miniconda3\envs\transformers\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 20/11076 [00:05<48:22,  3.81it/s] 

{'loss': 2.3298, 'grad_norm': 1.4622387886047363, 'learning_rate': 9.98194293968942e-05, 'epoch': 0.01}


  0%|          | 41/11076 [00:10<38:54,  4.73it/s]

{'loss': 2.2671, 'grad_norm': 1.5158990621566772, 'learning_rate': 9.963885879378839e-05, 'epoch': 0.01}


  1%|          | 60/11076 [00:14<39:42,  4.62it/s]

...

100%|██████████| 11076/11076 [48:34<00:00,  4.82it/s]c:\Users\User\miniconda3\envs\transformers\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                     
100%|██████████| 11076/11076 [51:59<00:00,  4.82it/s]

{'eval_loss': 1.7369529008865356, 'eval_rouge1': 0.1455863378848119, 'eval_rouge2': 0.08963037515095953, 'eval_rougeL': 0.13247995053071152, 'eval_rougeLsum': 0.13959237030808536, 'eval_runtime': 205.5742, 'eval_samples_per_second': 7.983, 'eval_steps_per_second': 3.994, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
100%|██████████| 11076/11076 [52:01<00:00,  3.55it/s]

{'train_runtime': 3121.212, 'train_samples_per_second': 14.193, 'train_steps_per_second': 3.549, 'train_loss': 2.0111337708583075, 'epoch': 3.0}


TrainOutput(global_step=11076, training_loss=2.0111337708583075, metrics={'train_runtime': 3121.212, 'train_samples_per_second': 14.193, 'train_steps_per_second': 3.549, 'total_flos': 462242880798720.0, 'train_loss': 2.0111337708583075, 'epoch': 3.0})

## inference

In [9]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
last_checkpoint = "./results/checkpoint-11076"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint).to("cuda")
finetuned_tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

input_text = "Please answer this medical related question: What is (are) Parasites - Lice - Body Lice ?"
input_ids = finetuned_tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = finetuned_model.generate(
    input_ids,
    max_length=200,
    min_length=40,
    repetition_penalty=2.0
)
answer = finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)
from textwrap import fill

print(fill(answer, width=100))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Body lice are a rare, common parasite that can cause serious health problems. They may be found in
the skin or hair and also in other parts of the body. The most commonly known type of body lice is
sexy-like (fever) lice. It usually occurs in people with no history of their infestation. In some
cases, it is not treated by an insecticide.
